### Practise

**Connecting a New Module**

Our control system, built on the application factory, is ready for expansion. Our task is to create a new, independent module (Blueprint) for geological drone diagnostics and connect it to our main application.

**Preparation:**
We will use the functional "Assembly Line" application from the previous lesson (Lesson 7) - and extend it.

**Assignment:**

- **"Assembly Line" structure:**
- processor_project/
- wsgi.py
- app/
    - _ _ init _ _.py
    - config.py
    - routes.py
    - **drone_diagnostics/**
        - **_ _ init _ _.py**
        - **routes.py**

1.  **Create a package for the Blueprint:** inside the `app/` folder, create a `drone_diagnostics/` folder.

2.  **Create the Blueprint:** in `app/drone_diagnostics/`, create an `__init__.py` file - here, define a new Blueprint named `'drone'`.

3.  **Prepare routes for the Blueprint:** in `app/drone_diagnostics/`, create a `routes.py` file - here, create at least one route specific to drones - for example, `@drone_bp.route('/ping')`, which returns the text `"Successful connection with drone diagnostics!"`.

4.  **Register the Blueprint:** in the application factory in `app/__init__.py`, import the `drone_bp` blueprint, set its prefix to `url_prefix='/drone'`, and register it on the `app` instance.

5.  **Verify functionality:** run the application and verify that the routes work, and that the new route is available at `http://127.0.0.1:5000/drone/ping`.

In [ ]:
# app/config.py

SECRET_KEY = 'a-secret-key-for-the-processor'
DEBUG = True

In [ ]:
# app/drone_diagnostics/__init__.py

from flask import Blueprint


drone_bp = Blueprint('drone', __name__)

from . import routes

In [ ]:
# app/drone_diagnostics/routes.py

from . import drone_bp

@drone_bp.route('/ping')
def ping():
    return "Successfull connection with drone diagnostics!"

In [ ]:
# app/routes.py

def register_routes(app):
    @app.route("/")
    def index():
        return "<h1>Atmospheric Processor Monitor</h1><p>Status: Online</p>"

    @app.route("/data")
    def get_data():
        return {"pressure": "102.5 kPa", "oxygen_level": "20.9%"}

In [ ]:
# app/__init__.py

from flask import Flask
from . import config
from . import routes
from .drone_diagnostics import drone_bp

def create_app():
    app = Flask(__name__)
    app.config.from_object(config)

    routes.register_routes(app)
    app.register_blueprint(drone_bp, url_prefix='/drone')

    return app

In [ ]:
# wsgi.py

from app import create_app

app = create_app()

if __name__ == "__main__":
    app.run()

### **Project (Homework): Extending the Core with Specialized Modules**

**Mission:** In the last mission, we successfully rebuilt the core of our system into a robust foundation with an application factory. Now it's time for the next step: all our communication protocols and control panels, which are still in a single `routes.py` file, will be divided into specialized, independent modules (Blueprints).


**Target application structure:**
- my_project/
    - wsgi.py
    - app/
        - _ _ init _ _.py
        - database.py
        - models/
            - _ _ init _ _.py
            - crewmember.py
        - data/
            - _ _ init _ _.py
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
        - **main/** Blueprint pro globální / společné routy (podbalíček "main")
            - **_ _ init _ _.py**
            - **routes.py** - routy pro hlavní / společné rozhraní
            - **templates/** - šablony k routám
        - **crew/** Blueprint pro routy s posádkou (podbalíček "crew")
            - **_ _ init _ _.py**
            - **routes.py** - routy pro posádku
            - **forms.py** - formuláře
            - **templates/** - šablony k routám
        - **api/** Blueprint pro API routy (podbalíček "api")
            - **_ _ init _ _.py**
            - **routes.py** - API routy
        - /instance
            - my_database.db

- The file with our **SQLite DB** is in the default **instance** folder next to our application, which is the **root** folder.


**1. Division into Specialized Modules (Blueprints):**
* **Main Bridge (`main`):** Create the `app/main/` **package** for the Blueprint - move all general routes (`/`, `/mission_briefing`, `/target_asteroid`) and their templates here - from the global `/templates` to the `main` blueprint `/templates`, and perform the corresponding imports into `__init__.py`.
* **Crew Management Section (`crew`):** Create the `app/crew/` **package** - move everything related to the crew here: routes (`/crew_members`, `/crew_members/<int:id>`, `/onboarding`), the `NewCrewMemberForm` form, and their templates - from the global `/templates` to the `crew` blueprint `/templates`, and perform the corresponding imports into `__init__.py`.
* **Automated Data Channel (`api`):** Create the `app/api/` package - move all your API endpoints here, and perform the corresponding imports into `__init__.py`.

**2. Reconfiguring Internal Systems (Fixing Imports):**
* By moving files into new packages, their relative paths have changed - we must go through all the files we have moved (`routes.py`, `forms.py`), and **fix the `import` statements** so that they correspond to the new structure.
* *Tip: An import that was previously `from .database import db` (from `app/routes.py`) must now, for example, in `app/crew/routes.py`, be updated to `from ..database import db`, because we need to go one level up.*

**3. Recalibrating Navigation Links (`url_for`):**
* After moving the routes to Blueprints, their internal names (endpoints) have changed (e.g., from `'homepage'` to `'main.homepage'`).
* We need to perform a refactoring = go through all our templates and our code and **update all uses of `url_for()`** so that they use the new, full endpoint names.
* *Tip: For links *within* a single Blueprint, it is possible to use a relative path with a dot (`.`).*

**4. Final Module Integration:**
* In the application factory (`app/__init__.py`), import and **register** all three new Blueprints.

**5. Systems Check and Functionality Verification:**
* Verify that the specific templates in the Blueprints correctly inherit from the shared `base.html` in the main `app/templates/` folder.
* Verify that all links and redirects throughout the application still work correctly, even though they are now split between different Blueprints.

In [ ]:
# app/config/configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True 
    STATIC_FOLDER = 'static' 
    TEMPLATES_FOLDER = 'templates' 
    SECRET_KEY = 'my_secret_something' 

    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' 
    SQLALCHEMY_TRACK_MODIFICATIONS = False 

In [ ]:
# app/__init__.py

from .config.configuration import Config
from flask import Flask
from .database import db  # import 'db' object
from .data import my_crew # import data
from .models import CrewMember  # import table models
from .main import main_bp # import blueprint object from 'main'
from .crew import crew_bp # import blueprint object from 'crew'
from .api import api_bp # import blueprint object from 'api'



def app_factory():

    app = Flask(__name__)

    # CONFIGURACION
    app.config.from_object(Config) 

    # PLUGIN INITIALIZATION - extensions for "app" object
    db.init_app(app) 

    # REGISTRATION (Blueprints)
    app.register_blueprint(main_bp) # register Main BP
    app.register_blueprint(crew_bp) # register Crew BP
    app.register_blueprint(api_bp, url_prefix="/api") # register API BP, with prefix "/api"
    
    # APP CONTEXT
    with app.app_context(): 
        db.create_all() 
        if not CrewMember.query.first(): 
            for crew_member in my_crew:
                    new_crew_member = CrewMember(
                    id=crew_member["id"], 
                    name=crew_member["name"], 
                    specialization=crew_member["specialization"], 
                    status=crew_member["status"])
            db.session.add(new_crew_member)
            db.session.commit()

    return app

## /models

In [ ]:
# app/models/__init__.py

from .crewmember import CrewMember

In [ ]:
# app/models/crewmember.py

from ..database import db 


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

## /data

In [ ]:
# app/data/__init__.py

from .asteroid import my_asteroid
from .crew import my_crew

In [ ]:
# data/asteroid

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}

In [ ]:
# data/crew

my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

## /main

In [ ]:
# app/main/__init__.py

from flask import Blueprint

# Blueprint 'main'
main_bp = Blueprint('main', __name__, template_folder='templates')

from . import routes # import AFTER Blueprint

In [ ]:
# app/main/routes.py

from flask import Flask, render_template
from ..data import my_asteroid
from . import main_bp # import blueprint object


@main_bp.route("/")
def homepage():
    print("homepage called")
    return render_template("homepage.html")

@main_bp.route("/mission_briefing")
def mission_briefing():
    return render_template("mission_briefing.html")

@main_bp.route("/target_asteroid")
def target_asteroid():
    print("target_asteroid called")
    return render_template("target_asteroid.html", my_asteroid=my_asteroid)

## /crew

In [ ]:
# app/crew/__init__.py

from flask import Blueprint

# Blueprint 'main'
crew_bp = Blueprint('crew', __name__, template_folder='templates')

from . import routes # import AFTER Blueprint

In [ ]:
# app/crew/routes.py

from flask import render_template, redirect, url_for
from .forms import NewCrewMemberForm
from ..models import CrewMember
from ..database import db
from . import crew_bp # import blueprint object


@crew_bp.route("/crew_members")
def crew_members():
    my_crew = db.session.query(CrewMember).all()
    return render_template("crew_members.html", my_crew=my_crew)

@crew_bp.route("/crew_members/<int:id>")
def crew_members_details(id):
    crew_member = db.session.query(CrewMember).get(id)
    if crew_member:
        return render_template("crew_members_details.html", crew_member=crew_member)
    return "Crew member not found"
    
@crew_bp.route("/onboarding", methods=["GET", "POST"])
def onboarding():
    form = NewCrewMemberForm()
    if form.validate_on_submit():
        db.session.add(CrewMember(
            name=form.name.data,
            specialization=form.specialization.data,
            status="Pending.."
        ))
        db.session.commit()
        return redirect(url_for(".crew_members")) # relative BP path
    return render_template("onboarding.html", form=form)

In [ ]:
# app/crew/forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class NewCrewMemberForm(FlaskForm):
    name = StringField(label="Name", validators=[validators.DataRequired()])
    specialization = StringField(label="Specialization", validators=[validators.DataRequired()])
    
    submit = SubmitField(label="Submit new crew member")

## /api

In [ ]:
# app/api/__init__.py

from flask import Blueprint

# Blueprint 'api'
api_bp = Blueprint('api', __name__)

from . import routes # import AFTER Blueprint

In [ ]:
# app/api/routes.py

from flask import Flask, jsonify
from ..database import db
from ..models import CrewMember
from ..data import my_asteroid
from . import api_bp # import blueprint object

   
@api_bp.route("crew_members")
def api_crew_members():
    all_crew_members = db.session.query(CrewMember).all()
    return jsonify(list({"id":member.id, "name":member.name, "specialization":member.specialization, "status":member.status} for member in all_crew_members)), 200
    
@api_bp.route("target_asteroid")
def api_target_asteroid():
    return jsonify(my_asteroid), 200

## /app

In [ ]:
# app/database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

## /models

In [ ]:
# app/models/__init__.py

from .crewmember import CrewMember

In [ ]:
# app/models/crewmember.py

from ..database import db # Importujeme 'db' objekt


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

---

In [ ]:
# wsgi.py

from app import app_factory


app = app_factory()

if __name__ == "__main__":
    app.run(host='0.0.0.0')

## /templates

In [ ]:
<!-- header.html -->

<nav>
    <ul>
        {# global blueprint 'main' #}
        <li><a href="{{ url_for('main.homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('main.mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('main.target_asteroid') }}">The Asteroid</a></li>

        {# global blueprint 'crew' #}
        <li><a href="{{ url_for('crew.crew_members') }}">Our Crew</a></li>
        <li><a href="{{ url_for('crew.onboarding') }}">New Recruits Onboarding</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> 
    
</head>

<body>
    {% include 'header.html' %}
    {% block page_content %} Content {% endblock page_content %} 
    {% include 'footer.html' %} 
</body>

</html>

## /main/templates

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} 

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %} 

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %} 

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/> 
{% endblock page_content %}

## /crew/templates

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} 

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %} 

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}

        {# relative path - within the BP #}
        <p><a href="{{ url_for('.crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
        
    {% endfor %}
{% endblock page_content %}

In [ ]:
<!-- onboarding.html -->

{% extends "base.html" %} 

{% block page_title %} Onboarding {% endblock page_title %}

{% block page_content %}
<form method = "POST" action = "{{ url_for('.onboarding') }}">  <!-- relative path - within the BP -->
    <p>New crew member onboarding form</p>
    {{ form.hidden_tag() }} 

    <p>{{ form.name.label }} {{ form.name(size=30) }}</p>
    <p>{{ form.specialization.label }} {{ form.specialization(size=30) }}</p>

    <p>{{ form.submit() }}</p>
</form>
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom